
主要包括 列表、字典的一些基本操作。更基本的就没放上去了

### 1. Unpacking a Sequence into Seperate Variables

常见的是 `a, b = (1, 2)` 这种类型，但是如果等式右边长度不确定，左边只有有限个变量，怎么办？

使用 `*` 标记

In [2]:

def drop_first_last(grades):
    first, *middle, last = grades
    return sum(middle) / len(middle)

assert drop_first_last([1,2, 2, 2, 2, 100]) == 2

### 2. Keeping the Last N Items

常常会遇见这样的问题： 保存最近的 N 个历史记录。

第一反应是用链表。操作上稍稍有些麻烦。

另一个思路是用固定数组，使用2个指针表示头和尾，循环操作

建议使用 `collections.deque`， 实现方式（cpython）见 <https://github.com/python/cpython/blob/master/Modules/_collectionsmodule.c> ，大致思路： 使用2个指针表示头和尾，循环操作。

In [8]:
from collections import deque

class History(object):
    def __init__(self, size):
        self.store = deque(maxlen=size)
        
    def append(self, item):
        self.store.append(item)
    
    def show(self):
        print(list(self.store))

history = History(5)

for i in range(10):
    history.append(i)
    history.show()

[0]
[0, 1]
[0, 1, 2]
[0, 1, 2, 3]
[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]


### 3. Finding the Largest or Smallest N Items

找最大或最小的N个元素。 

第一反应是先排序， 然后取前N个或后N个。 这样做法的复杂度是 $O(n \log n)$。 

可以使用 `heapq` 结构，即优先级队列结构，可以找到最大或最小的 N 个元素。 最大堆和最小堆结构。

回顾一下堆排序算法：

1. max-heapify， 保持最大堆特性： $O(\log n)$
2. build-max-heap， 从数组构建堆结构：从中间节点向上，逐步调用max-heapify，直观上需要 $O(n \log n)$，实际上渐进上界是 $O(n)$
3. heapsort， 堆排序，因为根节点最大，所以不断的抽取根节点、调整堆结构：  $O(n \log n)$

堆结构有个重要的作用是可以做为优先级队列。 在构建堆结构（$O(n)$）之后，取前k 个最大的只需要 $O(k \log n)$ 的时间

In [7]:
import heapq

def nlargest(items, n, key=None):
    return heapq.nlargest(n, items, key=key)

def nsmallest(items, n, key=None):
    return heapq.nsmallest(n, items, key=key)

nums = [1, 2, 3, 4, 5, 6, 7, 8]
print(nlargest(nums, 3))
print(nsmallest(nums, 3))

[8, 7, 6]
[1, 2, 3]


### 4. 实现优先级队列

用 `heapq.heappush` 就行了。 heapq的原理是使用原列表， 维护一种堆结构。所以不需要额外空间。 默认使用最小堆

注意下面例子里面， item 实际是一个三元组 `(-priority, self._index, item)` ， 所以比较的时候，先比较 `-priority`，优先级顺序，然后是 `index`，先进先出， 最后是 `item`

In [9]:
import heapq

class PriorityQueue:
    def __init__(self):
        self._queue = []
        self._index = 0
    
    def push(self, item, priority):
        heapq.heappush(self._queue, (-priority, self._index, item))
        self._index += 1
    
    def pop(self):
        return heapq.heappop(self._queue)[-1]
    
q = PriorityQueue()
q.push('foo', 1)
q.push('bar', 5)
q.push('spam', 4)
q.push('grok', 1)

print(q.pop())
print(q.pop())
print(q.pop())
print(q.pop())

bar
spam
foo
grok


### 5. Mapping Keys to Multiple Values in a Dictionary

在字典里面，一个键对应多个值。 实际上可以用列表保存多个值。所以每个键对应一个列表（或者集合）。 这里涉及初始化的问题。通常的做法是：

```python
d = {}
d.setdefault('a', []).append(1)
```

实际上可以用 `collections.defaultdict`

In [11]:
from collections import defaultdict

d = defaultdict(list)
d['a'].append(1)
d

defaultdict(list, {'a': [1]})

### 6. Keeping Dictionaries in Order

这个问题是，如何保证字典的插入顺序？

所以字典的键值不再是一个集合，而是一个列表（同时需要保证去重）。

python提供 `collections.OrderedDict` 。

注意 `OrderedDict` 的存储空间是普通字典的 2倍。 所以视需求而定。

In [12]:
from collections import OrderedDict

d = OrderedDict()
d['foo'] = 1
d['bar'] = 2
d['spam'] = 3
d['grok'] = 4

for key in d:
    print(key, d[key])

foo 1
bar 2
spam 3
grok 4


### 7. Finding Commonalities in Two Dictionaries

找到两个字典中相同的键或者值。

注意一个字典的键 `d.keys()` 或者值 `d.values()` 或者键值对 `d.items()` 都是 集合类型的对象。可以用集合操作。

In [15]:
a = {'x': 1, 'y': 2, 'z': 3}
b = {'w': 10, 'x': 11, 'y': 2}

print(type(a.keys()), type(a.values()), type(a.items()))
print(a.keys() & b.keys())
print(a.keys() - b.keys())

<class 'dict_keys'> <class 'dict_values'> <class 'dict_items'>
{'y', 'x'}
{'z'}


### 8. Removing Duplicates from a Sequence while Maintaining Order

给定一个序列数据， 移除重复元素的同时保证原来的顺序。

如果不要求保证原来的顺序，可以直接用 `set`。 但是这也要求 数据项是 hashable的。

如果要保证顺序，需要遍历序列，并且维护一个集合，用于保存“见过”的数据项。 对于不可hashable的，需要提供一个 `key` 方法，用于映射到某个可比较的值，用于辨识数据项是否一致。


In [18]:

def dedupe(items, key=None):
    seen = set()
    for item in items:
        val = item if key is None else key(item)
        if val not in seen:
            yield item
            seen.add(val)

a = [{'x': 1, 'y': 2}, {'x': 1, 'y': 3}, {'x': 1, 'y': 2}, {'x': 1, 'y': 2}, {'x': 2, 'y': 4}, ]

list(dedupe(a, key=lambda d: d['x']))


[{'x': 1, 'y': 2}, {'x': 2, 'y': 4}]

### 9. Naming  a Slice

有一些分片操作，用一些 magic number，不利于维护。 其实可以命名这些分片。 用 `slice`

In [22]:
######### 01234567890123456789012345678901234567890123456789
record = '.....................100           .....513.25.         ....'
cost = int(record[21:24]) * float(record[40:46])

SHARES = slice(21, 24)
PRICE = slice(40, 46)

assert (int(record[SHARES]) * float(record[PRICE]) == cost)

### 10. 统计一个数组中出现次数最多的元素

用一个字典统计计数就可以了。

更直接的，用 `collections.Counter`


In [25]:
words = """A Counter is a dict subclass for counting hashable objects. It is an unordered collection where elements are stored as dictionary keys and their counts are stored as dictionary values. Counts are allowed to be any integer value including zero or negative counts. 
The Counter class is similar to bags or multisets in other languages.""".lower().replace('.', '').split()


from collections import Counter
word_counts = Counter(words)
print(word_counts.most_common(3))


[('is', 3), ('are', 3), ('counts', 3)]


### 11. sort 操作中 ，key的选择和用法

有时候需要对字典列表或者其他不可比较的数据类型进行排序。这时候需要给出 `key`

有几种方式：

1. lambda函数
2. `operator.itemgetter`，这是用于字典类型某个键值的获取
3. `operator.attrgetter`，这是用于某个对象的某个属性的获取

### 12. group操作

对字典列表或对象列表，按照某个键值或者属性，对其进行分组。 用到 `itertools.groupby`

### 13. filter 操作

有几种形式：

1. `[x for x in a if x > 0]` 这种表达式形式
2. `filter(lambda x: x > 0, a)` 
3. `itertools.compress`，这种实际上是用第二个序列的结果（boolean值），应用到第一个序列上去。

In [27]:
from itertools import compress

a = [1, 2, 3, 4, 5]
b = [x % 2 for x in a]
bb = [x == 0 for x in b]

print(a)
print(bb)
print(list(compress(a, bb)))
print(list(compress(bb, a)))

[1, 2, 3, 4, 5]
[False, True, False, True, False]
[2, 4]
[False, True, False, True, False]


### 14.  Combining Multiple Mappings into a Single Mapping

比如有几种映射关系 map1, map2, map3， 如果想串接起来，组成一个合成的映射关系，可以用 `collections.ChainMap`

挺有意思的，但是不知道什么地方用。

`ChainMap(map1, map2, map3)` 表示 map3 -> map2 -> map1 这样的映射顺序。前面的映射会被后面的覆盖

In [31]:
a = {'x': 1, 'y': 3}
b = {'y': 2, 'z': 4}

from collections import ChainMap
c = ChainMap(a, b)
print(c['x'])  # 1
print(c['y'])  # 3  from a  这里可以理解成反向排序， a是最后一个映射关系。
print(c['z'])  # 4
print(c)

1
3
4
ChainMap({'x': 1, 'y': 3}, {'y': 2, 'z': 4})
